# The undirected version of the Chinese postman problem
Alicia Fernández Martínez 905092



In graph theory, the Chinese Postman Problem (CPP) addresses a question:
"Given a weighted graph, what is the shortest closed path that traverses each edge of the graph at least once?"

This problem, named after a Chinese mathematician, Mei-Ko Kwan, models real-world scenarios such as mail delivery routes, garbage collection paths, or street-sweeping plans. However, when the graph is undirected, the solution involves specific adaptations, which makes it a variant called the Undirected Chinese Postman Problem (UCPP).


In [13]:
from itertools import combinations # Combinations for node pairing
import heapq # heapq for priority queue in Dijkstra
import numpy as np

# Hungarian Algorithm class
class HungarianAlgorithmGraph:
    def __init__(self, cost_matrix):
        self.cost_matrix = cost_matrix
        self.n = len(cost_matrix)
        self.U = set(range(self.n))
        self.V = set(range(self.n))
        self.matching = {}
        self.labels_u = [0] * self.n
        self.labels_v = [0] * self.n
        self.slack = [float('inf')] * self.n

    def initialize_labels(self):
        for u in range(self.n):
            self.labels_u[u] = max(self.cost_matrix[u])

    def find_augmenting_path(self, u, visited_u, visited_v, parent):
        visited_u[u] = True
        for v in range(self.n):
            if visited_v[v]:
                continue
            delta = self.labels_u[u] + self.labels_v[v] - self.cost_matrix[u][v]
            if delta == 0:
                visited_v[v] = True
                if v not in self.matching or self.find_augmenting_path(self.matching[v], visited_u, visited_v, parent):
                    self.matching[v] = u
                    return True
            else:
                self.slack[v] = min(self.slack[v], delta)
        return False

    def update_labels(self, visited_u, visited_v):
        delta = min(self.slack[v] for v in range(self.n) if not visited_v[v])
        for u in range(self.n):
            if visited_u[u]:
                self.labels_u[u] -= delta
        for v in range(self.n):
            if visited_v[v]:
                self.labels_v[v] += delta
            else:
                self.slack[v] -= delta

    def augment(self):
        for u in range(self.n):
            visited_u = [False] * self.n
            visited_v = [False] * self.n
            self.slack = [float('inf')] * self.n
            while not self.find_augmenting_path(u, visited_u, visited_v, {}):
                self.update_labels(visited_u, visited_v)
                visited_u = [False] * self.n
                visited_v = [False] * self.n

    def solve(self):
        self.initialize_labels()
        self.augment()
        result = [(self.matching[v], v) for v in self.matching]
        optimal_cost = sum(self.cost_matrix[u][v] for u, v in result)
        return result, optimal_cost

# Find Eulerian tour
def find_eulerian_tour(graph):
    degree = {node: len(neighbors) for node, neighbors in graph.items()}
    odd_degree_nodes = [node for node, deg in degree.items() if deg % 2 == 1]

    tour = []
    stack = [next(iter(graph))]
    while stack:
        node = stack[-1]
        if graph[node]:
            next_node, _ = graph[node].pop()
            graph[next_node].remove((node, _))
            stack.append(next_node)
        else:
            tour.append(stack.pop())
    return tour

# Chinese Postman Problem
def chinese_postman(graph):
    degree = {node: len(neighbors) for node, neighbors in graph.items()}
    odd_degree_nodes = [node for node, deg in degree.items() if deg % 2 == 1]
    if not odd_degree_nodes:
        return find_eulerian_tour(graph.copy()), 0
    pairs = list(combinations(odd_degree_nodes, 2))
    distances = {(u, v): single_source_shortest_paths(graph, u)[v] for u, v in pairs}

    cost_matrix = np.zeros((len(odd_degree_nodes), len(odd_degree_nodes)))
    node_map = {node: i for i, node in enumerate(odd_degree_nodes)}
    for i, u in enumerate(odd_degree_nodes):
        for j, v in enumerate(odd_degree_nodes):
             if i != j:
                if (u,v) in distances:
                    cost_matrix[i][j] = distances[(u,v)]
                elif (v,u) in distances:
                    cost_matrix[i][j] = distances[(v,u)]
                else:
                    cost_matrix[i][j] = float('inf')

    hungarian = HungarianAlgorithmGraph(cost_matrix)
    assignment, _ = hungarian.solve()

    match = []
    for u_index, v_index in assignment:
        u = odd_degree_nodes[u_index]
        v = odd_degree_nodes[v_index]
        match.append((u, v))

    for u, v in match:
        graph[u].append((v, distances[(u, v)] if (u, v) in distances else distances[(v,u)]))
        graph[v].append((u, distances[(u, v)] if (u, v) in distances else distances[(v,u)]))

    tour = find_eulerian_tour(graph.copy())
    additional_cost = sum(distances[(u, v)] if (u, v) in distances else distances[(v,u)] for u, v in match)
    return tour, additional_cost

# Create graph from edges
def graph_from_edges(edges, directed=False):
    graph = {}
    for u, v, weight in edges:
        if u not in graph:
            graph[u] = []
        if v not in graph:
            graph[v] = []
        graph[u].append((v, weight))
        if not directed:
            graph[v].append((u, weight))
    return graph

# Dijkstra for shortest paths
def single_source_shortest_paths(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]:
            continue
        for neighbor, weight in graph[current_node]:
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    return distances

# Examples of usage

In [14]:
#1
graph_weighted = graph_from_edges([(0, 1, 2), (1, 2, 3), (2, 3, 1), (3, 4, 4), (4, 5, 2), (5, 0, 3)], directed=False)
postman_tour, added_cost = chinese_postman(graph_weighted)
print("Chinese Postman Tour:", postman_tour)
print("Additional cost:", added_cost)

#2
graph_weighted_2 = graph_from_edges([
    (0, 1, 4), (1, 2, 6), (2, 3, 5), (3, 4, 7), (4, 0, 3), (1, 3, 2)
], directed=False)
postman_tour_2, added_cost_2 = chinese_postman(graph_weighted_2)
print("Chinese Postman Tour:", postman_tour_2)
print("Additional cost:", added_cost_2)

#3 (large example)
graph_weighted_large = graph_from_edges([
    (0, 1, 10), (0, 2, 15), (0, 3, 20),
    (1, 2, 35), (1, 4, 25), (1, 5, 30),
    (2, 3, 40), (2, 6, 50), (3, 7, 45),
    (4, 5, 5), (4, 6, 10), (5, 7, 20),
    (6, 7, 25), (6, 8, 30), (7, 8, 35),
    (8, 9, 40), (7, 9, 20), (5, 9, 15)
], directed=False)

postman_tour_large, added_cost_large = chinese_postman(graph_weighted_large)

print("Chinese Postman Tour (Large Graph):", postman_tour_large)
print("Additional cost (Large Graph):", added_cost_large)

Chinese Postman Tour: [0, 1, 2, 3, 4, 5, 0]
Additional cost: 0
Chinese Postman Tour: [3, 2, 0, 1, 3, 1, 3, 4, 0]
Additional cost: 4
Chinese Postman Tour (Large Graph): [9, 7, 4, 6, 3, 1, 0, 2, 1, 5, 4, 6, 8, 7, 5, 9, 7, 4, 7, 3, 9, 3, 0, 8, 0]
Additional cost (Large Graph): 330
